In [50]:
from tensorflow import keras 
import tensorflow as tf
import numpy as np

In [2]:
imdb = keras.datasets.imdb

In [3]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 2s 0us/step


In [15]:
print("trainData shape: {}".format(train_data.shape))
print(train_data[0])

trainData shape: (25000,)


AttributeError: 'list' object has no attribute 'shape'

In [16]:
for idx in range(10):
    print(len(train_data[idx]))

218
189
141
550
147
43
123
562
233
130


In [8]:
word_dict = imdb.get_word_index()

word_dict = {k: (v+3) for k,v in word_dict.items()}
word_dict["<PAD>"] = 0
word_dict["<START>"] = 1
word_dict["<UNK>"] = 2
word_dict["<UNUSED>"] = 3

reverse_dict = dict(zip(list(word_dict.values()), list(word_dict.keys())))

1646592/1641221 [==============================] - 0s 0us/step


In [11]:
def decode_text(target):
    return ' '.join([reverse_dict.get(i, '?') for i in target])

decode_text(train_data[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

In [17]:
ragged_tensor = tf.ragged.constant(train_data)


In [37]:
ragged_tensor.shape[0]

25000

In [63]:
tf_data = tf.data.Dataset.from_tensor_slices(ragged_tensor)
mapped = tf.map_fn(pad_seq_in_word, tf_data)

ValueError: Attempt to convert a value (<TensorSliceDataset shapes: (None,), types: tf.int32>) with an unsupported type (<class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'>) to a Tensor.

In [62]:
tf_data = tf.data.Dataset.from_tensor_slices(ragged_tensor).map(pad_seq_in_word)



AttributeError: in converted code:

    <ipython-input-61-27f9737c24b0>:12 pad_seq_in_word  *
        list_urls = urls.to_list()

    AttributeError: 'Tensor' object has no attribute 'to_list'


In [59]:
padded_data =  pad_sequences(train_data, maxlen=200, padding='post')
padded_data

array([[   5,   25,  100, ...,   19,  178,   32],
       [   1,  194, 1153, ...,    0,    0,    0],
       [   1,   14,   47, ...,    0,    0,    0],
       ...,
       [   1,   11,    6, ...,    0,    0,    0],
       [   1, 1446, 7079, ...,    0,    0,    0],
       [   1,   17,    6, ...,    0,    0,    0]], dtype=int32)

In [60]:
tf_data_padded = tf.data.Dataset.from_tensor_slices(padded_data).batch(500)


In [61]:

from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

def pad_map_fn(urls):
    return tf.map_fn(pad_seq_in_word_v2, urls)

def pad_seq_in_word_v2(urls):
    return pad_sequences(urls, padding='post')

def pad_seq_in_word(urls):
    max_d1 = 200
    list_urls = urls.to_list()
    pad_urls = np.zeros((len(list_urls), max_d1))
    for d0 in range(len(urls)): 
        url = urls[d0]
        for d1 in range(len(url)): 
            if d1 < max_d1: 
                pad_urls[d0,d1] = url[d1]
    return pad_urls 

'''
pad_seq가 사용되는 경우는 emb_mode가 4,5일 때이다. char-level word인 경우 
urls에 x_char가 들어가는데... x_chars 는 URL의 단어별로 쪼개고, 단어를 ngram단위로 쪼개고 난 다음에, id로 mapping 시킨것. 

이 함수가 하고자 하는 바는, word 단위에서 다시 ngram으로 쪼개서, 그 ngram string이랑, 해당 vector를 적는것 
max_d1 = .max_len_words => 한 URL에 최대로 들어갈 수 있는 단어의 갯수 (200) 
max_d2 = max_len_subwords => 한 단어에 최대로 들어갈 수 있는 단어의 갯수 (20)
'''
def pad_seq(urls, max_d1=0, max_d2=0, embedding_size=128): 
    
    # 정해져있지 않은경우, 찾는과정. 
    if max_d1 == 0 and max_d2 == 0: 
        for url in urls: 
            if len(url) > max_d1: 
                max_d1 = len(url) 
            for word in url: 
                if len(word) > max_d2: 
                    max_d2 = len(word) 
                    
    # pad_idx 는 URL 내 단어의 character들을 받는다. 근데 그 character는 embedding size의 vec로 표현된다.
    # 따라서 pad_idx의 차원은 URL 갯수 * URL내 최대 단어갯수 * 단어 내 최대 글자갯수 * 글자의 embedding vector 수 
    pad_idx = np.zeros((len(urls), max_d1, max_d2, embedding_size))
    pad_urls = np.zeros((len(urls), max_d1, max_d2))
    
    # 해당 character가 존재함을 표시하기 위해서 [1,1,1,1,1] 과 같은 vector로 표현한다. 
    pad_vec = [1 for i in range(embedding_size)]
    for d0 in range(len(urls)): 
        url = urls[d0]
        for d1 in range(len(url)): 
            
            if d1 < max_d1: 
                # word는 ngram으로 이루어진 단어 => emb_mode가 4,5 일때.. 
                word = url[d1]
                
                # 한 단어 내에서 character 나, ngram에 접근할 때는. 한 단어가 최대 몇개의 character로 이뤄지는지를 알아야함.
                for d2 in range(len(word)): 
                    if d2 < max_d2: 
                        # 한 단어의 character를 넣는다. . 
                        pad_urls[d0,d1,d2] = word[d2]  # 그 string으로 들어간다. 
                        
                        # pad
                        pad_idx[d0,d1,d2] = pad_vec  # 해당 string이 embedding vector로 들어간다. 
                        
    # pad_urls 는 단                
    return pad_urls, pad_idx